In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from parsing.src.web.agents.md_scene_agent import _default_parse_scenes

In [4]:
from pathlib import Path
from tqdm.auto import tqdm
import os
import json

/Users/egorbykov/Desktop/Работа/python/python_venvs/Hacatons_3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def batch_convert_to_markdown(input_dir, output_dir=None, recursive=True):
    """
    Конвертирует все .docx и .pdf файлы в директории в markdown формат.

    Args:
        input_dir: путь к директории с файлами
        output_dir: путь к директории для сохранения .md файлов (если None, создается рядом)
        recursive: искать файлы рекурсивно
        file_types: список типов файлов для обработки (['docx', 'pdf'] по умолчанию)

    Returns:
        dict: словарь с количеством обработанных файлов по типам
    """
    input_path = Path(input_dir)

    if output_dir is None:
        output_path = input_path / "json"
    else:
        output_path = Path(output_dir)

    output_path.mkdir(parents=True, exist_ok=True)


    file_types = ['md']

    # Находим все файлы указанных типов
    all_files = []
    for file_type in file_types:
        if recursive:
            files = list(input_path.rglob(f"*.{file_type}"))
        else:
            files = list(input_path.glob(f"*.{file_type}"))
        all_files.extend(files)

    # Исключаем временные файлы (начинающиеся с ~$)
    all_files = [f for f in all_files if not f.name.startswith('~$')]

    converted_files = {'md': 0}
    errors = []

    for file_path in tqdm(all_files, desc="Конвертация файлов"):
        try:
            # Создаем относительный путь для сохранения структуры директорий
            relative_path = file_path.relative_to(input_path)
            md_file_name = relative_path.with_suffix('.json')
            md_file_path = output_path / md_file_name


            # Создаем необходимые поддиректории
            md_file_path.parent.mkdir(parents=True, exist_ok=True)

            print(f"Обрабатываю: {file_path.name}...", end=" ")
            with open(file_path, "r", encoding="utf-8") as f:
                list_screens = _default_parse_scenes(f.read())


            if md_file_path:
                with open(md_file_path, "w", encoding="utf-8") as f:
                    json.dump(list_screens, f, ensure_ascii=False, indent=4)
                print(f"✓ JSON файл сохранен: {md_file_name}")

            converted_files['md'] += 1

            print("✓")
        except Exception as e:
            error_msg = f"✗ Ошибка при обработке {file_path.name}: {e}"
            print(error_msg)
            errors.append(error_msg)

    print(f"\nОбработано файлов:")
    print(f"  - MD: {converted_files['md']}")
    print(f"  - Всего: {sum(converted_files.values())}/{len(all_files)}")

    if errors:
        print(f"\nОшибок: {len(errors)}")

    return converted_files

In [6]:
batch_convert_to_markdown('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md', output_dir='/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/json', recursive=True)

Конвертация файлов: 100%|██████████| 59/59 [00:00<00:00, 595.89it/s]

Обрабатываю: ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_6с_13.09_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_6с_13.09_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_2С_15.08_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_2С_15.08_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_3С_05.09_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_3С_05.09_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_5с_13.09_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_5с_13.09_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_4с_10.09_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_4с_10.09_ФИНАЛ.json
✓
Обрабатываю: ПТ_С3_Д_22.06.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Политех (с таблицей)/ПТ_С3_

{'md': 59}

In [8]:
def find_files_with_text(input_dir, search_text, file_types=['md'], recursive=True):
    input_path = Path(input_dir)

    # Находим все файлы
    all_files = []
    for file_type in file_types:
        if recursive:
            files = list(input_path.rglob(f"*.{file_type}"))
        else:
            files = list(input_path.glob(f"*.{file_type}"))
        all_files.extend(files)

    # Исключаем временные файлы
    all_files = [f for f in all_files if not f.name.startswith('~$')]

    found_files = []
    for file_path in tqdm(all_files, desc="Поиск текста"):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                if search_text in f.read():
                    found_files.append(file_path)
        except Exception as e:
            print(f"Ошибка: {file_path.name}: {e}")

    return found_files

# Использование
search_text = '1-4. ИНТ. КВАРТИРА БОКОВА.КУХНЯ НОЧЬ 1.'
found = find_files_with_text('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md', search_text)

print(f"Найдено: {len(found)} файлов")
for f in found:
    print(f"  {f}")

Поиск текста: 100%|██████████| 59/59 [00:00<00:00, 6512.04it/s]

Найдено: 1 файлов
  /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Фишер (с таблицей)/1/ФИШЕР 1 сери __16.05.md
